In [52]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from matplotlib.patches import Ellipse
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage


from sklearn import metrics

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
import time

In [53]:
#datasets_93172_883723_hfi_cc_2019.csv

human_file = '/Users/mehrunisaqayyum/Downloads/datasets_93172_883723_hfi_cc_2019.csv' 
human_df = pd.read_csv(human_file)
human_df

,year,ISO_code,countries,region,hf_score,hf_rank,hf_quartile,pf_rol_procedural,pf_rol_civil,pf_rol_criminal,...,ef_regulation_business_adm,ef_regulation_business_bureaucracy,ef_regulation_business_start,ef_regulation_business_bribes,ef_regulation_business_licensing,ef_regulation_business_compliance,ef_regulation_business,ef_regulation,ef_score,ef_rank
0,2017,ALB,Albania,Eastern Europe,7.84,38,1,6.7,4.5,4.7,...,6.3,6.7,9.7,4.1,6,7.2,6.7,7.8,7.67,30
1,2017,DZA,Algeria,Middle East & North Africa,4.99,155,4,-,-,-,...,3.7,1.8,9.3,3.8,8.7,7,5.7,5.4,4.77,159
2,2017,AGO,Angola,Sub-Saharan Africa,5.4,151,4,-,-,-,...,2.4,1.3,8.7,1.9,8.1,6.8,4.9,5.7,4.83,158
3,2017,ARG,Argentina,Latin America & the Caribbean,6.86,77,2,7.1,5.8,4.3,...,2.5,7.1,9.6,3.3,5.4,6.5,5.7,5.6,5.67,147
4,2017,ARM,Armenia,Caucasus & Central Asia,7.42,54,2,-,-,-,...,4.6,6.2,9.9,4.6,9.3,7.1,6.9,7.5,7.7,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,2008,AUS,Australia,Oceania,8.73,4,1,8.8,7.2,7.2,...,3.7,4.6,9.9,8.6,7.3,8.8,7.1,8.5,8.18,6
1616,2008,DNK,Denmark,Western Europe,8.73,4,1,9.4,7.9,8.7,...,4.7,7.4,9.7,9.4,9.7,8.5,8.2,8.6,7.98,9
1617,2008,CHE,Switzerland,Western Europe,8.83,3,1,-,-,-,...,5.5,7.8,9.3,8.8,8.3,9.3,8.1,8.5,8.35,4
1618,2008,NZL,New Zealand,Oceania,8.94,2,1,8.7,7.6,7.9,...,4.5,4.5,10,9.6,9.8,9.2,7.9,8.8,8.46,3


In [43]:
human_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1620 entries, 0 to 1619
Columns: 120 entries, year to ef_rank
dtypes: int64(1), object(119)
memory usage: 1.5+ MB


In [44]:
human_df.columns

Index(['year', 'ISO_code', 'countries', 'region', 'hf_score', 'hf_rank',
       'hf_quartile', 'pf_rol_procedural', 'pf_rol_civil', 'pf_rol_criminal',
       ...
       'ef_regulation_business_adm', 'ef_regulation_business_bureaucracy',
       'ef_regulation_business_start', 'ef_regulation_business_bribes',
       'ef_regulation_business_licensing', 'ef_regulation_business_compliance',
       'ef_regulation_business', 'ef_regulation', 'ef_score', 'ef_rank'],
      dtype='object', length=120)

In [45]:
human_df.columns.tolist()
#another way is to print(df.columns)

['year',
 'ISO_code',
 'countries',
 'region',
 'hf_score',
 'hf_rank',
 'hf_quartile',
 'pf_rol_procedural',
 'pf_rol_civil',
 'pf_rol_criminal',
 'pf_rol',
 'pf_ss_homicide',
 'pf_ss_disappearances_disap',
 'pf_ss_disappearances_violent',
 'pf_ss_disappearances_organized',
 'pf_ss_disappearances_fatalities',
 'pf_ss_disappearances_injuries',
 'pf_ss_disappearances',
 'pf_ss_women_fgm',
 'pf_ss_women_inheritance_widows',
 'pf_ss_women_inheritance_daughters',
 'pf_ss_women_inheritance',
 'pf_ss_women',
 'pf_ss',
 'pf_movement_domestic',
 'pf_movement_foreign',
 'pf_movement_women',
 'pf_movement',
 'pf_religion_estop_establish',
 'pf_religion_estop_operate',
 'pf_religion_estop',
 'pf_religion_harassment',
 'pf_religion_restrictions',
 'pf_religion',
 'pf_association_association',
 'pf_association_assembly',
 'pf_association_political_establish',
 'pf_association_political_operate',
 'pf_association_political',
 'pf_association_prof_establish',
 'pf_association_prof_operate',
 'pf_asso

In [5]:
human_df.shape

(1620, 120)

In [6]:
print(human_df.median())

year    2012.5
dtype: float64


### Observation
There are 1,620 observations and 120 indices/features. 

# Data Cleaning
Use df.drop([*insert column names*], axis=1).fillna(df.drop([*insert column names*], axis=1).median())
maybe need human_df = pd.to_numeric(human_df, errors='coerce')

In [54]:
human_df = human_df.replace('-', None, regex=True)

In [55]:
human_df.drop(['year','ISO_code','countries','region'], axis=1).fillna(human_df.drop(
    ['year','ISO_code', 'countries', 'region'], axis=1).median(),inplace = True)

In [56]:
#Define 'X'
X = human_df.iloc[:,5:119]

In [58]:
X

,hf_rank,hf_quartile,pf_rol_procedural,pf_rol_civil,pf_rol_criminal,pf_rol,pf_ss_homicide,pf_ss_disappearances_disap,pf_ss_disappearances_violent,pf_ss_disappearances_organized,...,ef_regulation_labor,ef_regulation_business_adm,ef_regulation_business_bureaucracy,ef_regulation_business_start,ef_regulation_business_bribes,ef_regulation_business_licensing,ef_regulation_business_compliance,ef_regulation_business,ef_regulation,ef_score
0,38,1,6.7,4.5,4.7,5.3,9.1,10,10,10,...,6.9,6.3,6.7,9.7,4.1,6,7.2,6.7,7.8,7.67
1,155,4,6.7,4.5,4.7,3.8,9.5,10,9.5,5,...,5.4,3.7,1.8,9.3,3.8,8.7,7,5.7,5.4,4.77
2,151,4,6.7,4.5,4.7,3.4,8.1,10,9.7,7.5,...,5.4,2.4,1.3,8.7,1.9,8.1,6.8,4.9,5.7,4.83
3,77,2,7.1,5.8,4.3,5.7,8,5,10,7.5,...,5.1,2.5,7.1,9.6,3.3,5.4,6.5,5.7,5.6,5.67
4,54,2,7.1,5.8,4.3,4.9,9,10,10,7.5,...,6.4,4.6,6.2,9.9,4.6,9.3,7.1,6.9,7.5,7.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,4,1,8.8,7.2,7.2,7.8,9.5,10,10,10,...,8.5,3.7,4.6,9.9,8.6,7.3,8.8,7.1,8.5,8.18
1616,4,1,9.4,7.9,8.7,8.6,9.6,10,10,10,...,7.5,4.7,7.4,9.7,9.4,9.7,8.5,8.2,8.6,7.98
1617,3,1,9.4,7.9,8.7,8,9.7,10,10,10,...,7.9,5.5,7.8,9.3,8.8,8.3,9.3,8.1,8.5,8.35
1618,2,1,8.7,7.6,7.9,8.1,9.5,10,10,10,...,8.5,4.5,4.5,10,9.6,9.8,9.2,7.9,8.8,8.46


In [59]:
human_df.applymap(np.isreal).all(1)

0       False
1       False
2       False
3       False
4       False
        ...  
1615    False
1616    False
1617    False
1618    False
1619    False
Length: 1620, dtype: bool

In [60]:
human_df.applymap(lambda x: isinstance(x, (int, float)))

,year,ISO_code,countries,region,hf_score,hf_rank,hf_quartile,pf_rol_procedural,pf_rol_civil,pf_rol_criminal,...,ef_regulation_business_adm,ef_regulation_business_bureaucracy,ef_regulation_business_start,ef_regulation_business_bribes,ef_regulation_business_licensing,ef_regulation_business_compliance,ef_regulation_business,ef_regulation,ef_score,ef_rank
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1616,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1617,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1618,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [61]:
human_df2 = (human_df.drop('year','ISO_code','countries','region', axis=1).join(human_df['year','ISO_code','countries','region'].apply(pd.to_numeric, errors='coerce')))
human_df2[human_df2['year','ISO_code','countries','region'].notnull().all(axis=1)]

TypeError: drop() got multiple values for argument 'axis'

human_df2 = human_df.drop(['year','ISO_code','countries','region'], axis=1).fillna(human_df.drop(['year','ISO_code', 'countries', 'region'], axis=1).median())
#human_df = human_df.fillna(human_df.median([:,5:119])

In [39]:
human_df2

,hf_score,hf_rank,hf_quartile,pf_rol_procedural,pf_rol_civil,pf_rol_criminal,pf_rol,pf_ss_homicide,pf_ss_disappearances_disap,pf_ss_disappearances_violent,...,ef_regulation_business_adm,ef_regulation_business_bureaucracy,ef_regulation_business_start,ef_regulation_business_bribes,ef_regulation_business_licensing,ef_regulation_business_compliance,ef_regulation_business,ef_regulation,ef_score,ef_rank
0,7.84,38,1,6.7,4.5,4.7,5.3,9.1,10,10,...,6.3,6.7,9.7,4.1,6,7.2,6.7,7.8,7.67,30
1,4.99,155,4,-,-,-,3.8,9.5,10,9.5,...,3.7,1.8,9.3,3.8,8.7,7,5.7,5.4,4.77,159
2,5.4,151,4,-,-,-,3.4,8.1,10,9.7,...,2.4,1.3,8.7,1.9,8.1,6.8,4.9,5.7,4.83,158
3,6.86,77,2,7.1,5.8,4.3,5.7,8,5,10,...,2.5,7.1,9.6,3.3,5.4,6.5,5.7,5.6,5.67,147
4,7.42,54,2,-,-,-,4.9,9,10,10,...,4.6,6.2,9.9,4.6,9.3,7.1,6.9,7.5,7.7,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,8.73,4,1,8.8,7.2,7.2,7.8,9.5,10,10,...,3.7,4.6,9.9,8.6,7.3,8.8,7.1,8.5,8.18,6
1616,8.73,4,1,9.4,7.9,8.7,8.6,9.6,10,10,...,4.7,7.4,9.7,9.4,9.7,8.5,8.2,8.6,7.98,9
1617,8.83,3,1,-,-,-,8,9.7,10,10,...,5.5,7.8,9.3,8.8,8.3,9.3,8.1,8.5,8.35,4
1618,8.94,2,1,8.7,7.6,7.9,8.1,9.5,10,10,...,4.5,4.5,10,9.6,9.8,9.2,7.9,8.8,8.46,3


In [48]:
#for x in range(5,119):
 #   human_df = human_df.apply(lambda x: x.fillna(x.median()),axis=0)

### Standardize

In [57]:
#Standarizing the features
scaler = StandardScaler()
X_std = scaler.fit_transform(X)
print (X_std)

ValueError: could not convert string to float: '-'